In [133]:
from collections import defaultdict

import numpy as np
import pandas as pd

In [134]:
df = pd.read_csv('responses.csv')
answers = pd.read_csv('answers.csv')

In [135]:
a_dict = dict()
questions = answers.loc[:,"Example Scenario in Survey"]
correct = answers.loc[:,"Correct Answer"]
for i in range(0, len(questions)):
    a_dict[questions[i]] = correct[i]

In [136]:
analysis = dict()
valcounts = list(df.columns.values.tolist())
for i in range(7, len(valcounts)):
    correct = 0
    incorrect = 0
    answers = df[list(df.columns.values.tolist())[i]].value_counts().keys().tolist()
    counts = df[list(df.columns.values.tolist())[i]].value_counts().tolist()
    if a_dict.get(valcounts[i], 42) == 42: continue
    for j in range(len(answers)):
        if answers[j] == a_dict[valcounts[i]]:
            correct = counts[j]
        else:
            incorrect += counts[j]
    analysis[valcounts[i]] = (correct, incorrect)

In [137]:
print("Section 1: \n")
for val in analysis:
    if "user/device physical location to area greater" in val:
        print("Section 2: \n")
    (correct, incorrect) = analysis[val]
    print(val + ": ")
    print("Correct= " + str(correct) + ", Incorrect= " + str(incorrect))
    print("Rate= " + str((correct/(correct+incorrect))))
    print("\n")

Section 1: 



In [138]:
df = pd.read_csv('responses.csv')

descriptionsDict = dict()
start = df.columns.get_loc("Q109")
end = df.columns.get_loc("Q101")
for i in range(start, end + 1, 1):
    key = df.columns[i]
    descriptionsDict[key] = df.iat[0, i]

In [139]:
# only keep responses that can pass the attention check
df = df[df["Q135"] == "Neither"]
df = df[df["Q137"] == "Health and Fitness"]

In [140]:
typeDict = dict()
datatypeQIds = {"Q109", "Q108", "Q128", "Q16", "Q17", "Q129", "Q19", "Q26", "Q20", "Q122", "Q22", "Q21", "Q23", "Q132", "Q133", "Q24", "Q123", "Q125", "Q25", "Q127", "Q27", "Q130"}
section1 = ['Data Collection', 'Data Sharing', 'Both', 'Neither']
section2 = ['Location', 'Personal Information', 'Financial Information', 'Health and Fitness', 'Messages', 'Photos or Videos', 'Audio Files', 'Files and docs', 'Calendar', 'Contacts', 'App Activity', 'Web Browsing', 'App Information and Performance', 'Device or Other Identifiers', 'None of the Above']
for i in range(start, end + 1, 1):
    key = df.columns[i]
    if key in datatypeQIds:
        entry = 1
    else:
        entry = 2
    typeDict[key] = entry
answerDict = defaultdict(str)
correctAnswerDict = defaultdict(str)
indexGivenDict = defaultdict(str)
indexExpectedDict = defaultdict(str)
for i in range(start, end+1, 1):
    for j in range(0, len(df.iloc[:, 0]), 1):
        participantId = df.iat[j, df.columns.get_loc("Q162")]
        questionId = df.columns[i]
        rowId = (participantId, questionId)
        answer = df.iat[j, i]
        if isinstance(answer, str):
            answer = answer.strip()
        if "(Optional)" in descriptionsDict[questionId]: continue
        else:
            if answer not in section1 and answer not in section2:
                continue
            answerDict[rowId] = answer
            correctAnswerDict[rowId] = a_dict[descriptionsDict[questionId]]
            if typeDict[questionId] == 1:
                index_given = section1.index(answer)
                index_expected = section1.index(a_dict[descriptionsDict[questionId]])
            else:
                index_given = section2.index(answer)
                index_expected = section2.index(a_dict[descriptionsDict[questionId]])
            indexGivenDict[rowId] = index_given
            indexExpectedDict[rowId] = index_expected
conditionDict = dict()
for j in range(0, len(df.iloc[:, 0]), 1):
    conditionDict[df.iat[j, df.columns.get_loc("Q162")]] = df.iat[j, -1]

ValueError: 'Personal info' is not in list

In [ ]:
rowIds = []
for i in range(0, len(df.iloc[:, 0]), 1):
    participantId = df.iat[i, df.columns.get_loc("Q162")]
    for j in range(start, df.columns.get_loc("Q101")+ 1):
        questionId = df.columns[j]
        if "(Optional)" in descriptionsDict[questionId]: continue
        else: rowIds.append((participantId, questionId))
rowIds = sorted(rowIds)

In [ ]:
results = pd.DataFrame()
participantIds = [p for p, q in rowIds]
questionIds = [q for p, q in rowIds]
descriptions = [descriptionsDict[q] for q in questionIds]
types = [typeDict[q] for q in questionIds]
answers = [answerDict[(p, q)] for p, q in zip(participantIds, questionIds)]
correctAnswers = [correctAnswerDict[(p, q)] for p, q in zip(participantIds, questionIds)]
conditions = [conditionDict[p] for p in participantIds]
givenIndices = [indexGivenDict[(p, q)] for p, q in zip(participantIds, questionIds)]
expectedIndices = [indexExpectedDict[(p, q)] for p, q in zip(participantIds, questionIds)]

results['Question ID'] = questionIds
results['Participant ID'] = participantIds
results['Description'] = descriptions
results['Answer'] = answers
results['Correct Answer'] = correctAnswers
results['Question Type'] = types
results['Prompt Condition'] = conditions
results['Answer Index'] = givenIndices
results['Correct Answer Index'] = expectedIndices

# filter out rows with empty answers (i.e., not selected in the study)
results = results[results["Answer"] != ""]

results['Correctness'] = (results['Answer'] == results['Correct Answer']).apply(lambda v: 1 if v else 0)
results['Order Difference'] =\
    (results['Answer Index'] - results['Correct Answer Index']).apply(lambda v: 0 if v == 0 else 1 if v > 0 else -1)

print(results)

In [ ]:
results.to_csv("output.csv")